Ce notebook vise à traiter les datasets concernant les matchs de tennis, les organiser et faire des modifications/néttoyages nécessaires selon le besoin 

1.Importation des librairies nécessaires 

In [1]:
import pandas as pd 

In [5]:
#Lecture des datasets 
df1 = pd.read_csv('Aust_Open_champs_69_21.csv')
df1.head(5)

,Year,Champion,Seed_Champion,Ctry_Champion,Runner-up,Seed_Runner-up,Ctry_Runner-up,Score in the Final
0,1969,Rod Laver,1,AUS,Andrés Gimeno,9,ESP,"6–3, 6–4, 7–5"
1,1970,Arthur Ashe,4,USA,Dick Crealy,12,AUS,"6–4, 9–7, 6–2"
2,1971,Ken Rosewall,2,AUS,Arthur Ashe,3,USA,"6–1, 7–5, 6–3"
3,1972,Ken Rosewall,2,AUS,Malcolm Anderson,8,AUS,"7–6(7–2), 6–3, 7–5"
4,1973,John Newcombe,2,AUS,Onny Parun,12,NZL,"6–3, 6–7, 7–5, 6–1"


In [6]:
#Lecture des datasets 
df2 = pd.read_csv('Aust_Open_champs_69_21_wom.csv')
df2.head(5)

,Year,Champion,Seed_Champion,Ctry_Champion,Runner-up,Seed_Runner-up,Ctry_Runner-up,Score in the Final
0,1969,Margaret Court,2,AUS,Billie Jean King,1,USA,"6–4, 6–1"
1,1970,Margaret Court,1,AUS,Kerry Melville,2,AUS,"6–1, 6–3"
2,1971,Margaret Court,1,AUS,Evonne Goolagong,2,AUS,"2–6, 7–6(7–0), 7–5"
3,1972,Virginia Wade,2,GBR,Evonne Goolagong,1,AUS,"6–4, 6–4"
4,1973,Margaret Court,1,AUS,Evonne Goolagong,2,AUS,"6–4, 7–5"


In [8]:
#Fusionner df1 et df2 
df1 = pd.read_csv('Aust_Open_champs_69_21.csv')
df2 = pd.read_csv('Aust_Open_champs_69_21_wom.csv')


merged_df = pd.concat([df1, df2], ignore_index=True)
merged_df = merged_df.reset_index(drop=True)

merged_df.to_csv('fusion_aust_open_champs.csv', index=False)
merged_df

,Year,Champion,Seed_Champion,Ctry_Champion,Runner-up,Seed_Runner-up,Ctry_Runner-up,Score in the Final
0,1969,Rod Laver,1,AUS,Andrés Gimeno,9,ESP,"6–3, 6–4, 7–5"
1,1970,Arthur Ashe,4,USA,Dick Crealy,12,AUS,"6–4, 9–7, 6–2"
2,1971,Ken Rosewall,2,AUS,Arthur Ashe,3,USA,"6–1, 7–5, 6–3"
3,1972,Ken Rosewall,2,AUS,Malcolm Anderson,8,AUS,"7–6(7–2), 6–3, 7–5"
4,1973,John Newcombe,2,AUS,Onny Parun,12,NZL,"6–3, 6–7, 7–5, 6–1"
...,...,...,...,...,...,...,...,...
101,2017,Serena Williams,2,USA,Venus Williams,13,USA,"6-4, 6-4"
102,2018,Caroline Wozniacki,2,DEN,Simona Halep,1,ROU,"7-6(7-2), 3-6, 6-4"
103,2019,Naomi Osaka,4,JPN,Petra Kvitova,8,CZE,"7-6(7-2), 5-7, 6-4"
104,2020,Sofia Kenin,14,USA,Garbiñe Muguruza,0,ESP,"4-6, 6-2, 6-2"
